<small><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a> ©2021 onwards</font></small><hr style="margin:0;background-color:silver">

**[<font size=6>🌌Stellar</font>](https://www.kaggle.com/competitions/2oct23hse-stellar/rules)**. [**Instructions**](https://colab.research.google.com/drive/1riOGrE_Fv-yfIbM5V4pgJx4DWcd92cZr#scrollTo=ITaPDPIQEgXV) for running Colabs.

<small>**(Optional) CONSENT.** <mark>[ X ]</mark> We consent to sharing our Colab (after the assignment ends) with other students/instructors for educational purposes. We understand that sharing is optional and this decision will not affect our grade in any way. <font color=gray><i>(If ok with sharing your Colab for educational purposes, leave "X" in the check box.)</i></font></small>

In [ ]:
#from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if your kaggle.json is stored in Google Drive

In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
# !cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 2oct23hse-stellar        # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

cp: cannot stat '/content/drive/MyDrive/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, i

In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, os, tqdm, re, sys, cv2, skimage
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression as LR
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA, LinearDiscriminantAnalysis as LDA
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('display.max_columns', 20, 'display.precision', 2, 'display.max_rows', 4)

CPU times: user 263 ms, sys: 4.17 ms, total: 268 ms
Wall time: 292 ms


In [ ]:
df = pd.read_csv('XY_Stellar.csv'); df

,alpha,delta,u,g,r,i,z,run_ID,cam_col,field_ID,redshift,plate,MJD,fiber_ID,Class
0,11.64,21.28,26.28,26.15,24.05,18.87,19.00,8848,5,272,0.84,7740,56824,833,NaN
1,173.09,42.21,22.51,22.83,22.21,19.55,19.96,4156,3,486,0.81,9041,58067,428,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199998,131.31,44.27,24.07,24.64,21.63,19.20,19.03,7076,3,251,0.55,6014,56166,1021,G
199999,22.59,0.25,25.30,25.56,24.09,19.41,19.96,5164,4,511,1.26,9590,57969,878,G


In [ ]:
df.info()   # observe datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 15 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   alpha     200000 non-null  float64
 1   delta     200000 non-null  float64
 2   u         200000 non-null  float64
 3   g         200000 non-null  float64
 4   r         200000 non-null  float64
 5   i         200000 non-null  float64
 6   z         200000 non-null  float64
 7   run_ID    200000 non-null  int64  
 8   cam_col   200000 non-null  int64  
 9   field_ID  200000 non-null  int64  
 10  redshift  200000 non-null  float64
 11  plate     200000 non-null  int64  
 12  MJD       200000 non-null  int64  
 13  fiber_ID  200000 non-null  int64  
 14  Class     160000 non-null  object 
dtypes: float64(8), int64(6), object(1)
memory usage: 22.9+ MB


In [ ]:
# Change string labels to numbers in order of increasing size of the entity (Star < Quasi Star < Galaxy)
# df.Class = df.Class.apply(lambda C: -1 if C=='S' else 0 if C=='Q' else 1 if C=='G' else None)

In [ ]:
vX = df.query('Class!=Class').drop('Class', axis=1)  # slice a test sample
tXY = df.query('Class==Class')                       # slice training sample
tX, tY = tXY.drop('Class', axis=1), tXY.Class        # split into training I/O

In [ ]:
def ScatterCorrHist(df):
  def corrdot(*args, **kwargs):
    # credit: https://stackoverflow.com/questions/48139899
    corr_r = args[0].corr(args[1], 'pearson')
    corr_text = f"{corr_r:2.2f}".replace("0.", ".")
    ax = plt.gca();
    ax.set_axis_off();
    msz = abs(corr_r) * 5000   # marker size
    fsz = abs(corr_r) * 40 + 5 # font size
    ax.scatter([.5], [.5], msz, [corr_r], alpha=0.5, cmap='coolwarm', vmin=-1, vmax=1, transform=ax.transAxes)
    ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction", ha='center', va='center', fontsize=fsz)

  sns.set(style='white', font_scale=.8);
  g = sns.PairGrid(df, aspect=1, diag_sharey=False);
  g.fig.set_size_inches(20,10)
  g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color':'red'}, scatter_kws={'s':1});
  g.map_diag(sns.histplot, kde_kws={'color':'black'});
  g.map_upper(corrdot);
  g.fig.suptitle("Scatter plot, Correlations and histograms on diagonal", y=1);
  _ = plt.subplots_adjust(hspace=0.02, wspace=0.02);
  _ = plt.show();

# ScatterCorrHist(tXY.head(200))

In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=green size=40>

<strong><font color=green size=5>⏳Timed Green Playground (TGP): Your ideas, code, documentation, and timer START HERE!</font></strong>

<font color=green>Students: Keep all your definitions, code, documentation in <b>TGP</b>. Modifying any code outside of TGP incurs penalties.

<font color=green><h3><b>$\alpha$. Build polynomial features</b><h3>

In [ ]:
from sklearn.preprocessing import StandardScaler
from collections import Counter

np.random.seed(0)
vX0, vY0 = tX.iloc[140000:,:], tY[140000:] # subsample for model selection experiments
fX0, fY0 = tX.iloc[120000:140000,:], tY[120000:140000]

In [ ]:
dfcpy = df
df = df.drop('field_ID', axis=1)
df = df.drop('fiber_ID', axis=1)
df = df.drop('run_ID', axis=1)
df = df.drop('plate', axis=1)

In [ ]:
def k_QDA(df, l, r):
  # Subsampling
  vX = df.query('Class!=Class').drop('Class', axis=1)  # slice a test sample
  tXY = df.query('Class==Class')                       # slice training sample
  tX, tY = tXY.drop('Class', axis=1), tXY.Class        # split into training I/O


  tX0, tY0 = tX.iloc[l:r,:], tY[l:r]   # subsample for model selection experiments
  vX0, vY0 = tX.iloc[140000:,:], tY[140000:] # subsample for model selection experiments
  fX0, fY0 = tX.iloc[120000:140000,:], tY[120000:140000]

  # ModelPipeline
  scaler = StandardScaler()
  tX0 = scaler.fit_transform(tX0)
  vX0 = scaler.transform(vX0)
  vX  = scaler.transform(vX)
  fX0 = scaler.transform(fX0)

  poly = PolynomialFeatures(degree=3)    # add non-linear features (powers and interactions)
  tX0p = poly.fit_transform(tX0)  # create object on training set
  vX0p = poly.transform(vX0)      # apply the same object to test set
  vXp  = poly.transform(vX)       # apply the same object to test set
  fX0 = poly.transform(fX0)

  mQDA = QDA()
  mQDA = mQDA.fit(tX0p, tY0)

  # Predictions
  QDA_y_pred = mQDA.predict(vX0p)
  f_QDA_y_pred = mQDA.predict(fX0)
  f_QDA_y_kaggle = mQDA.predict(vXp)

  return QDA_y_pred, f_QDA_y_pred, f_QDA_y_kaggle

In [ ]:
def k_LR(df, l, r):
    # Subsampling
    vX = df.query('Class!=Class').drop('Class', axis=1)  # slice a test sample
    tXY = df.query('Class==Class')                       # slice training sample
    tX, tY = tXY.drop('Class', axis=1), tXY.Class        # split into training I/O

    tX0, tY0 = tX.iloc[l:r,:], tY[l:r]   # subsample for model selection experiments
    vX0, vY0 = tX.iloc[140000:,:], tY[140000:] # subsample for model selection experiments
    fX0, fY0 = tX.iloc[120000:140000,:], tY[120000:140000]

    # ModelPipeline
    scaler = StandardScaler()
    tX0 = scaler.fit_transform(tX0)
    vX0 = scaler.transform(vX0)
    vX  = scaler.transform(vX)
    fX0 = scaler.transform(fX0)

    poly = PolynomialFeatures(degree=3)    # add non-linear features (powers and interactions)
    tX0p = poly.fit_transform(tX0)  # create object on training set
    vX0p = poly.transform(vX0)      # apply the same object to test set
    vXp  = poly.transform(vX)       # apply the same object to test set
    fX0 = poly.transform(fX0)

    mlogreg = LR(random_state=0, C=25, max_iter=100)
    mlogreg = mlogreg.fit(tX0p, tY0)

    # Predictions
    logreg_y_pred = mlogreg.predict(vX0p)
    f_logreg_y_pred = mlogreg.predict(fX0)
    f_logreg_y_kaggle = mlogreg.predict(vXp)

    return logreg_y_pred, f_logreg_y_pred, f_logreg_y_kaggle

<font color=green><h3><b>$\beta$. Fit a model</b><h3>

In [ ]:
%%time
import warnings
warnings.filterwarnings('ignore')

m2, f_m2, f_log_reg_y_kaggle = k_LR(df, 0, 30000)
m3, f_m3, f_QDA_y_kaggle = k_QDA(df, 25000, 55000)

m5, f_m5, f_log_reg_y_kaggle2 = k_LR(df, 50000, 80000)

m4, f_m4, f_QDA_y_kaggle2 = k_QDA(df, 75000, 100000)


CPU times: user 35.1 s, sys: 11 s, total: 46.1 s
Wall time: 54.5 s


<font color=green><h3><b>$\gamma$. Make predictions</b><h3>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Encode the categorical labels into numerical values
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(vY0)
y_test = label_encoder.transform(fY0)

# One-hot encode the features
encoder = OneHotEncoder(sparse=False, dtype=np.int)
X_train = np.array([m2, m3, m4, m5])
X_train = encoder.fit_transform(X_train.T)
X_train = np.hstack((X_train, vX0))

X_test = np.array([f_m2, f_m3, f_m4, f_m5])
X_test = encoder.transform(X_test.T)
X_test = np.hstack((X_test, fX0))

# Standard scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Polynomial features
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Train a LogisticRegression model as an example
clf = LDA()
clf.fit(X_train_poly, y_train)

# Make predictions on the testing set
final_predictions_encoded = clf.predict(X_test_poly)

# Decode the predictions back to categorical labels
final_predictions = label_encoder.inverse_transform(final_predictions_encoded)

# Calculate accuracy
accuracy = accuracy_score(label_encoder.inverse_transform(y_test), final_predictions)
print("Accuracy:", accuracy)


LinearDiscriminantAnalysis()

Accuracy: 0.97015


In [ ]:
# Using the final classifier
X_test_kaggle = np.array([f_log_reg_y_kaggle, f_QDA_y_kaggle, f_log_reg_y_kaggle2, f_QDA_y_kaggle2])
X_test_kaggle = encoder.transform(X_test_kaggle.T)
X_test_kaggle = np.hstack((X_test_kaggle, vX))
X_test_kaggle = scaler.transform(X_test_kaggle)
X_test_kaggle = poly.transform(X_test_kaggle)

final_predictions_encoded = clf.predict(X_test_kaggle)

# Inverse transform predictions
final_predictions = label_encoder.inverse_transform(final_predictions_encoded)

# Kaggle csv
pY = pd.DataFrame(final_predictions, index=range(1,len(vX)+1), columns=['Class'])
df.Class.fillna('unknown').value_counts()   # distribution of all train labels
pY.value_counts()                           # distribution of predicted labels
ToCSV(pY, '🌌Baseline🐍')

G          95204
unknown    40000
S          34362
Q          30434
Name: Class, dtype: int64

Class
G        23917
S         8898
Q         7185
dtype: int64

<font color=green><h3><b>$\epsilon$. Documentation</b></h3></font>

<font color=green><h4><b>Task 1. Explain Decisions in Preprocessing Pipeline</b></h4></font>

<font color=green>
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc.</font>

<font color=green>

1. Why did you choose these elements? (Something in EDA, prior experience,...? Note: EDA is not required)
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

</font>

<b>Why did you choose these elements?</b>
<ul>
  <li> Some features such as ‘field_id’, ‘fiber_id’, ‘run_id’ and ‘plate’ were dropped from the dataset because they did not provide any useful data for modelling, this was understood after EDA. </li>
  <li> Subsampling is used for model selection, validation, and testing to ensure the model is evaluated effectively and to avoid overfitting </li>
  <li> In order to scale the functions of model to a range StandardScaler was used. </li>
  <li> Polynomial features of 3rd degree are added to model non-linear relationships in the model. </li>
</ul>

During the preprocessing stage we have tried EDA by plotting coloured histograms. Furthermore, for each class and for each feature we have plotted boxplots. The data showed that for the ID features (field_id, fiber_id, run_id and plate) the distribution for each class was the same. This meant that we could delete those classes without impacting our future classifications. In the context this makes sense since an ID does not usually determine the type of object that is named with this ID. Using the histograms we also managed to observe that the deviations throughout classes was mostly insignificant and the only feature which really helped to classify objects was the redshift (to determine whether an object is a star or a Quasar/Galaxy).
Our next idea was to use linear combinations of different features in order to determine some properties. We have read articles that g-u is sometimes used to classify spatial objects. Unfortunately, all our attempts led nowhere.

<b> How do you evaluate the effectiveness of these elements? </b>

In order to measure the effectiveness of the chosen methods, we should look at the accuracy of the model which is quite high, more than 96%. Hence, the chosen methods are extremely effective.
To evaluate the effectiveness of the changes made to our data we have used cross validation on different samples of the same dataset. If dropping a feature or adding a feature led to a higher mean accuracy on our test dataset we went forward with the change.

<b> What else have you tried that worked or didn't? </b>
<ul>
  <li> We tried to drop various other features, however, the chosen ones were the most useless. </li>
  <li> We also tried to use PCA, however, it didn’t work properly and decreased the result of the model in which it was used, QDA. </li>
  <li> Other function for scaling, MaxAbsScaler was also used but the StandardScaler gave better results for our model. </li>
</ul>

Firstly, we tried PCA in order to find linear directions that led to maximal variance and thereby reduce the number of features used. However, decreasing components led to a lower accuracy score. This effect was furthered when we tried to use PCA after dropping the unnecessary features.
Secondly, we used StandardScaler and the effect was significant for the overall performance of our linear regression model, however we could not observe a significant change in performance of either QDA or LDA. To monitor the performance of each classifier separately we created the functions (which was removed from the last version of our google colab), but we are mentioning this since it made checking different components of different classifiers much more ‘user-friendly’.
Third, we decided to go forward with polynomial features to catch the quadratic and the combined relationships within our features. We have also tried degree 4, but this led to overfitting since our test performance was significantly worse than the train performance.

<font color=green><h4><b>Task 2. Explain Decisions in Modeling Pipeline</b></h4></font>

<font color=green>
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful.

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements?
1. What else have you tried that worked or didn't?

</font>

<b> How did these decisions guide you in modelling? </b>

Our big idea was to use a number of classifiers since this could lead to different properties being caught by the different classifiers and thereby a higher accuracy. We got this idea because when we observed the confusion matrices for QDA and LogReg we realised that they were mispredicting differently and different classes. This meant we could somehow average the predictions of many classifiers. To do so, we trained multiple QDA and LogReg models on different samples of the data, then we used an LDA model with additional features consisting of the predictors of the different classifiers. This is how we got our final model pipeline.
We used a few models such as QDA and Logistic Regression on different parts of the dataset in order to predict results. After that, the obtained results were put into LDA, which output the final prediction.

<b> How do you evaluate the effectiveness of these elements? </b>

We think that it is extremely effective since we have managed to achieve an accuracy of 0.97015 % on our sample. Furthermore, we know that this kind of implementation is readily used in a RandomForest model, where many trees are taught on different data samples.

<b> What else have you tried that worked or didn't? </b>

Firstly, we have tried to use a single model for predictions. This could only achieve a satisfactory result of about 96.5% which we thought was not good enough. We tried to use many more models (3*7) but this went above the time limit, while achieving quite a satisfactory result. We also realised that the sample sizes were unbalanced and hence tried to use SMOTE to balance out the number of classes, but for some reason this has decreased our accuracy. Lastly, we used gridsearch cv to find the optimal. Lastly, we have also read on Kaggle that this dataset performs well with boosting models so it would be interesting to try these models on this dataset.

<font color=green><h3><b>$\zeta$. References</b></h3></font>

1. https://www.kaggle.com/code/ilialar/hyperparameters-tunning-with-hyperopt#Hyperopt (https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fwww.kaggle.com%2Fcode%2Filialar%2Fhyperparameters-tunning-with-hyperopt%23Hyperopt)
1. James, G., Witten, D., Hastie, T., Tibshirani, R., & Taylor, J. (2023). An introduction to statistical learning: With applications in Python. Springer.
1. Hastie, T. (2011). The elements of statistical learning, Hastie, 2nd edition. Springer.
1. https://en.wikipedia.org/wiki/Hyperparameter_optimization#Bayesian_optimization (https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fen.wikipedia.org%2Fwiki%2FHyperparameter_optimization%23Bayesian_optimization)

<font size=5>⌛</font> <strong><font color=green size=5>Do not exceed competition's runtime limit! Do not write code outside TGP</font></strong>
<hr color=green size=40>

In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 59 sec


## 💡**Starter Ideas**

1. Tune model hyperparameters
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest missclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.
1. Do scatter plots show piecewise linear shape? Can a separate linear model be used on each support, or can the pattern be linearized via transformations?
1. How are date/categorical features treated by the model? Is there a [better way](https://www.google.com/search?q=ways+to+encode+categorical+data) to encode these (perhaps, ordinal) features?
  1. E.g. you could replace codes (or groups of codes) with their frequencies, which may capture the implied "distance" or rarity between category levels.
  1. If encoding ordinal features with integers, should non-equidistant values be considered?
1. Learn astronomy domain and features: [🎦](https://www.youtube.com/results?search_query=Quasi-star), [quasi-star](https://en.wikipedia.org/wiki/Quasi-star), [star](https://en.wikipedia.org/wiki/Star), [galaxy](https://en.wikipedia.org/wiki/Galaxy), [📃](https://arxiv.org/abs/2112.02026)
